Сегодня мы рассмотрим библиотеку [einops](https://github.com/arogozhnikov/einops). Единственная функция этой библиотеки --- это упростить код и сделаеть его более читаемым. Все преобразованияо на формой и структурой тензора с помощью этой библиотеки можно задавать в виде строки с понятными обозначениями.

einops работает со следующими библиотеками: numpy, torch, jax, tensorflow, tinygrad, etc.

In [ ]:
from einops import rearrange, reduce
import torch

Сначала посмотрим, что могут делать базовые функции `rearrange` и `reduce`:

`rearrange` изменяет форму тензора согласно переданной строке

In [ ]:
rearrange(torch.randn(10, 3, 32, 32), "b c h w -> b h w c").shape  # аналог permute

torch.Size([10, 32, 32, 3])

In [13]:
rearrange(torch.randn(10, 3, 32, 32), "b c h w -> b 1 c h w").shape  # аналог unsqueeze(1)

torch.Size([10, 1, 3, 32, 32])

In [9]:
rearrange(torch.randn(10, 3, 32, 32), "b c h w -> b (c h w)").shape  # аналог flatten(1)

torch.Size([10, 3072])

Понятно, что вместо `b c h w` можно писать `batch channel height width`

`reduce` совмещает в себе изменение формы плюс уменьшение нужных размерностей

In [ ]:
reduce(torch.randn(10, 3, 32, 32), "b c (h 2) (w 2) -> b c h w", "mean").shape # аналог average pooling с ядром 2

torch.Size([10, 3, 16, 16])

Теперь посмотрим, как можно упрощать большие куски кода с помощью этой библиотеки. Для этого в файле `attention.py` написаны две реализации MultiHeadSelfAttetnion.

In [32]:
from attention import MHSA, MHSAEinops
from einops.layers.torch import Rearrange

In [35]:
torch.manual_seed(0)

mhsa = MHSA()
proj = torch.nn.Sequential(
    Rearrange("b c (h p1) (w p2) -> b (h w) (c p1 p2)", p1=2, p2=2),
    torch.nn.Linear(3 * 2 * 2, 512)
)
x = torch.randn(16, 3, 32, 32)
output_mhsa = mhsa(proj(x))

In [36]:
torch.manual_seed(0)

mhsa_einops = MHSAEinops()
output_mhsa_einops = mhsa(proj(x))

In [37]:
torch.allclose(output_mhsa, output_mhsa_einops)

True